In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

## Part 1: Import the Housing data and do feature transformations

In [2]:
df= pd.read_csv('house_price_full.csv')
df.head()

,bedrooms,sqft_living,price
0,3,1340,313000
1,5,3650,2384000
2,3,1930,342000
3,3,2000,420000
4,4,1940,550000


In [3]:
X = df.copy()
# Remove target
Y = X.pop('price')

# perform a scaler transform of the input data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# perform log transformation of target variable (For Sandeep: Is this needed?)
Y = np.log(Y)

In [4]:
df_scaled = pd.DataFrame(X)
df_scaled

,0,1
0,-0.433198,-0.753258
1,1.675735,1.457330
2,-0.433198,-0.188649
3,-0.433198,-0.121661
4,0.621269,-0.179079
...,...,...
494,0.621269,0.873582
495,1.675735,2.299459
496,-0.433198,-0.724549
497,-0.433198,-0.179079


In [5]:
Y

0      12.653958
1      14.684290
2      12.742566
3      12.948010
4      13.217674
         ...    
494    13.380102
495    13.764217
496    12.128111
497    12.721886
498    12.254863
Name: price, Length: 499, dtype: float64

## Part 2: Create Model Using `keras`

![](multiple_neurons.png)

In [6]:
from tensorflow import keras

In [7]:
model = keras.Sequential(
    [
        keras.layers.Dense(
            2, activation="sigmoid", input_shape=(X.shape[-1],)
        ),
        keras.layers.Dense(1, activation="linear")
    ]
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2)                 6         
                                                                 
 dense_1 (Dense)             (None, 1)                 3         
                                                                 
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


```python
def random_init_params():
    w1 = tf.Variable(tf.random.uniform((2, 2)))
    b1 = tf.Variable(tf.random.uniform((1, 2)))
    w2 = tf.Variable(tf.random.uniform((2, 1)))
    b2 = tf.Variable(tf.random.uniform((1, 1)))
    return w1,b1,w2,b2


def forward_prop(x, w1, b1, w2, b2):
    z1 = tf.matmul(x,w1) + b1
    h1 = tf.math.sigmoid(z1)
    z2 = tf.matmul(h1,w2) + b2
    h2 = z2
    return h2
```

In [8]:
model.compile(
    optimizer=keras.optimizers.SGD(), loss="mean_squared_error"
)

```python
def train(x, y, w1, b1, w2, b2):
    y_true = y
    with tf.GradientTape() as g:
        y_pred = forward_prop(x, w1, b1, w2, b2)

        # loss
        loss = 0.5*(y_true - y_pred)** 2
    
    #Gradient calculation  
    print("**************************************************")
    print("GRADIENTS")
    print("**************************************************")
    gw1, gb1, gw2, gb2 = g.gradient(loss, [w1, b1, w2, b2])
    print(" the gradient for 1st layer weights are:\n",gw1.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 2nd layer weights are:\n",gw2.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 1st layer bias are:\n",gb1.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 2nd layer bias are:\n",gb2.numpy())
    print("--------------------------------------------------")

    # Gradient descent:
    lr=0.2
    w1.assign_sub(lr*gw1)
    b1.assign_sub(lr*gb1) 
    w2.assign_sub(lr*gw2)
    b2.assign_sub(lr*gb2)
    print("**************************************************")
    print("NEW UPDATES")
    print("**************************************************")
    print(" the updated 1st layer weights are:\n",w1.numpy())
    print("--------------------------------------------------")
    print(" the updated 2nd layer weights are:\n",w2.numpy())
    print("--------------------------------------------------")
    print(" the updated 1st layer bias are:\n",b1.numpy())
    print("--------------------------------------------------")
    print(" the updated 2nd layer bias are:\n",b2.numpy())


    return w1, b1, w2, b2,loss

```

In [9]:
model.fit(X,Y.values,epochs=10,batch_size=32)

Epoch 1/10
16/16 [==============================] - 1s 5ms/step - loss: 123.1148
Epoch 2/10
16/16 [==============================] - 0s 2ms/step - loss: 40.3361
Epoch 3/10
16/16 [==============================] - 0s 1ms/step - loss: 9.2468
Epoch 4/10
16/16 [==============================] - 0s 1ms/step - loss: 1.8343
Epoch 5/10
16/16 [==============================] - 0s 2ms/step - loss: 0.4690
Epoch 6/10
16/16 [==============================] - 0s 1ms/step - loss: 0.2295
Epoch 7/10
16/16 [==============================] - 0s 1ms/step - loss: 0.1867
Epoch 8/10
16/16 [==============================] - 0s 2ms/step - loss: 0.1760
Epoch 9/10
16/16 [==============================] - 0s 2ms/step - loss: 0.1711
Epoch 10/10
16/16 [==============================] - 0s 1ms/step - loss: 0.1676


In [10]:
model.predict(X)[:,0]

16/16 [==============================] - 0s 3ms/step


array([12.908621 , 13.515926 , 13.022963 , 13.035246 , 13.1371   ,
       12.646327 , 12.73178  , 13.284498 , 13.104382 , 13.043293 ,
       12.982542 , 13.318774 , 13.089285 , 12.854776 , 13.390846 ,
       12.878833 , 12.955423 , 13.347551 , 12.91488  , 12.8745   ,
       13.075188 , 13.026835 , 12.990093 , 13.287873 , 13.061825 ,
       13.050621 , 13.093877 , 13.19059  , 13.373501 , 13.128664 ,
       13.052596 , 13.309246 , 13.111198 , 13.302774 , 13.41554  ,
       13.204485 , 12.630671 , 13.008581 , 13.028261 , 13.098415 ,
       12.957396 , 13.0339155, 12.6404915, 12.993835 , 12.707546 ,
       12.87015  , 13.030016 , 13.022963 , 12.893846 , 13.375556 ,
       13.284498 , 13.168453 , 13.143446 , 12.999407 , 13.244167 ,
       13.132834 , 13.188601 , 13.517403 , 12.904425 , 12.776941 ,
       13.375556 , 13.139486 , 13.287873 , 13.120293 , 13.318774 ,
       12.650193 , 12.97105  , 12.836937 , 12.904463 , 13.068766 ,
       13.253063 , 13.161203 , 13.206301 , 13.017612 , 12.9065